## <span style="color:gold">Algoritmi di Crittografia (2023/24)</span> 
## Notebook 4

In [ ]:
from IPython.display import HTML
HTML('<style>{}</style>'.format(open('/home/mauro/.jupyter/custom/custom.css').read()))

Latex definitions
$\DeclareMathOperator*{\prob}{prob}$
$\DeclareMathOperator*{\mod}{mod}$
$\DeclareMathOperator*{\ln}{ln}$
$\def\zn{\mathbf{Z}_n}$
$\def\zp{\mathbf{Z}_p}$
$\def\mcd{\mathrm{MCD}}$

## <span style="color:gold">Casualità e algoritmi probabilistici</span> 

### <span style="color:gold">Randomness (casualità)</span>
* Una risorsa di straordinaria importanza in molti algoritmi e applicazioni, in particolare proprio le applicazioni crittografiche, è rappresentata dalla disponibilità di <span style="color:gold">bit casuali</span>.
* In realtà questo è un modo sbrigativo ma comunemente utilizzato per definire <span style="color:gold">sequenze di bit generati casualmente, indipendenti e con probabilità uniforme</span>.
* Se $\lbrace X_i\rbrace_{i=1,2,..}$ è una tale sequenza, per ogni $i\geq 0$, 
<center>
    $\mathrm{prob}\lbrack X_i=0\rbrack = \mathrm{prob}\lbrack X_i=1\rbrack=\frac{1}{2}$
</center>
e
<center>
    $\mathrm{prob}\lbrack X_{i+1}=0|X_i=0\rbrack = \mathrm{prob}\lbrack X_{i+1}=0|X_i=1\rbrack=\mathrm{prob}\lbrack X_i=0\rbrack$
</center>
<center>
    $\mathrm{prob}\lbrack X_{i+1}=1|X_i=0\rbrack = \mathrm{prob}\lbrack X_{i+1}=1|X_i=1\rbrack=\mathrm{prob}\lbrack X_i=1\rbrack$
</center>
* Una sequenza di bit casuali è il punto terminale di un procedimento che ha inizio da una qualche <span style="color:gold">sorgente di casualità</span>, cioè un dispositivo logico o fisico in grado di produrre eventi casuali (misurabili) in un opportuno spazio.


#### <span style="color:cyan">Generatori hardware</span>
* Non è facile avere a disposizione sorgenti di "vera" casualità.
* Generatori veramente casuali sono tipicamente ottenibili mediante <span style="color:gold">dispositivi esterni al computer</span>, che sfruttano <span style="italic; color:gold">fenomeni fisici stocastici</span>.
* In generale, tali dispositivi sono realizzati intorno a tre componenti principali:
    1. un <span style="font-style: italic; color:gold">trasduttore</span>, che trasforma il rumore stocastico del fenomeno fisico utilizzato (ad esempio, rumore termico in un resistore o variazioni nell'ampiezza o nella fase di un oscillatore)  in segnali elettrici
    2. un <span style="font-style: italic; color:gold">amplificatore</span>, il cui scopo è di aumentare l'ampiezza dei segnali in modo che siano misurabili
    3. un <span style="font-style: italic; color:gold">convertitore analogico-digitale</span> che produce in output sequenze di bit
* La generazione attraverso questa via è però <span style="color:gold">costosa</span> e la quantità di bit casuali non è sempre adeguata alle esigenze delle applicazioni

#### <span style="color:cyan">Generatori pseudocasuali per applicazioni crittografiche (CSPRNG)</span>
* La disponibilità di sequenze di bit casuali in crittografia è richiesta in molteplici casi.
* Per gli scopi di questo corso è fondamentale comprendere l'importanza di generare <span style="color:gold">numeri casuali</span> che possano poi servire per la determinazione di chiavi in protocolli asimmetrici.
* I bit casuali necessari in applicazioni crittografiche vengono tipicamente ottenuti, nelle quantità richieste, mediante <span style="color:gold">procedimenti deterministici</span> a partire da <span style="color:gold">sequenze casuali</span> (in generale più corte).
* Le sequenze messe in questo modo a disposizione delle applicazioni sono dette <span style="font-style: italic; color:gold">pseudocasuali</span>.

#### <span style="color:cyan">CSPRNG in ambiente Linux</span>
* Le sequenze casuali sono ottenute, internamente al computer, da fenomeni <span style="color:gold">difficilmente predicibili</span>, anche se non sempre casuali.
* Fra questo possiamo ricordare i <span style="color:gold">movimenti del mouse</span>, la <span style="color:gold">frequenza di digitazione sulla tastiera</span>, il tempo che intercorre fra alcuni tipi di <span style="color:gold">interrupt</span>.
* Quanto più le sorgenti sono di <span style="color:gold">natura distinta</span> e le sequenze opportunamente <span style="color:gold">combinate</span>, tanto più i bit soddisfano le esigenze richieste dalle applicazioni crittografiche.
* I bit casuali <span style="color:gold">raccolti (collected)</span> dal sistema sono inseriti nel c.d. <span style="color:gold">entropy pool</span>, composto da max 4096 bit.
* Dal pool vengono prelevali bit casuali su richiesta delle applicazioni e in esso vengono inseriti bit casuali dal kernel.
* Il kernel tiene traccia del numero di bit casuali (o <span style="color:gold">bit di entropia</span>) presenti nel kernel.
* Tale quantità è conoscibile attraverso il comando indicato di seguito.

In [ ]:
!cat /proc/sys/kernel/random/entropy_avail

* Linux mette a disposizione interfacce per <span style="color:gold">due generatori</span>, che sono equivalenti quando l'<span style="color:gold">entropia è elevata</span> ma che <span style="color:gold">con poca entropia</span> si comportano differentemente.
* I generatori sono accessibili <span style="color:gold">come device</span> logici:
    1. <span style="color:gold">/dev/random</span>, che blocca il processo richiedente se l'entropia richiesta non è suffciente;
    2. <span style="color:gold">/dev/urandom</span>, che invece è <span style="color:gold">non blocking</span>.

In [ ]:
%%bash
dd if=/dev/random bs=8 count=1000 2>/dev/null  | base64

### <span style="color:magenta">CSPRNG in Python</span>

* In sistemi Linux, /dev/urandom è accessibile dal comando os.urandom di Python.
La chiamata di <span style="color:gold">os.urandom</span> restituisce il numero di byte casuali specificati come parametro.



In [ ]:
from os import urandom

In [ ]:
urandom(4)

In [ ]:
R = urandom(4)
for b in R:             
    print(bytes([b]))     # bytes(iterabile-di-interi) -> lista con altrettanti byte

* Possiamo anche visualizzare più convenientemente i byte come <span style="color:gold">sequenze esadecimali</span>. 

In [ ]:
from binascii import b2a_hex

In [ ]:
b2a_hex(R)

* Mettendo insieme i pezzi, possiamo creare un generatore casuale in $[0,1)$, che è poi il <span style="color:gold">generatore casuale fondamentale</span>

In [ ]:
def myrand():
    from os import urandom
    D = 2**(-64) 
    return int.from_bytes(urandom(8),'big')*D

In [ ]:
myrand()

### <span style="color:gold">Algoritmi probabilistici</span>
* Un algoritmo probabilistico è tale perché utilizza una qualche <span style="color:gold">sorgente di casualità</span>, che possiamo immaginare "tipo" <span style="color:gold">/dev/urandom</span>, cioè in grado di fornire una sequenza (teoricamente) illimitata di bit casuali, <span style="color:gold">indipendenti</span> e distribuiti in maniera <span style="color:gold">uniforme</span>.
* Algoritmi di questo tipo giocano un ruolo importante in crittografia e, in particolare (cioè in relazione a ciò che qui ci interessa), in <span style="color:gold">crittografia asimmetrica</span>.
* In questo caso, però, è conveniente immaginare che la sorgente produca direttamente <span style="color:gold">numeri casuali</span>, visto che le basi matematiche della crittografia asimmetrica risiedono proprio in teoria dai numeri.
* Per valutare la complessità degli algoritmi probabilistici che prenderemo in considerazione, si può ancora utilizzare il ben noto modello <span style="font-style: italic; color:gold">bit cost</span>, in cui ogni operazione aritmetico/logica ha un peso che dipende dal numero di bit degli operandi.
* Nella valutazione della bontà di un algoritmo è in generale necessario tenere conto anche della <span style="color:gold">quantità di bit casuali richiesti</span>, proprio perché i bit casuali non sono una risorsa illimitata.
* Se la casualità viene fornita in termini di numeri, piuttosto che di bit casuali, dobbiamo comunque valutare la lunghezza in bit dei numeri stessi.

### <span style="color:cyan">Algoritmi decisionali</span>
* &Egrave; detto <span style="font-style: italic; color:gold">decisionale</span> un algoritmo il cui output è binario: 0/1, True/False, Sì/No.
* Gli algoritmo probabilistici che vedremo saranno <span style="color:gold">solo di tipo decisionale</span>.
* L'elemento di casualità, nel comportamento visibile dell'algoritmo, può risiedere nel <span style="color:gold">risultato</span>, che può essere errato con una certa probabilità, o nel <span style="color:gold">tempo di calcolo</span>.
* A seconda dei due casi, gli algoritmi vengono definiti (di tipo) <span style="font-style: italic; color:gold">Monte Carlo</span> o (di tipo) <span style="font-style: italic; color:gold">Las Vegas</span>.
* Un algoritmo decisionale può essere visto come un <span style="color:gold">classificatore binario</span> (come avviene, ad esempio, nel contesto del <span style="color:gold">machine learning</span>).
* Ad esempio, dato un messaggio di posta elettronica, <span style="color:gold">decidere se è spam</span> (ovvero se appartiene all'insieme dei messaggi spam) o <span style="color:gold">non-spam</span>.

### <span style="color:cyan">Algoritmi Monde Carlo</span>
* Un algoritmo <span style="color:gold">probabilistico di tipo Monte Carlo</span> per un problema (decisionale) $P$, ha le caratteristiche indicate di seguito.
    1. Su input la cui risposta (esatta) è True (es. la mail è spam), <span style="color:gold">restituisce True con probabilità fissa</span> (indipendente dalla lunghezza dell'input) <span style="color:gold">$\epsilon>0$</span>.
    2. Su input $x\in P$ la cui risposta (esatta) è False, <span style="color:gold">restituisce False</span>.
* Per amore di precisione, ciò che abbiamo definito è un algoritmo denominato (con terminologia inglese, decisamente più sintetica) <span style="color:gold">Probability-bounded one-sided error Monte Carlo</span> .
    * <span style="color:gold">Probability-bounded</span> (sottinteso, "away from zero") perché si richiede che la probabilità di errore sia non solo positiva, ma che non tenda a zero al crescere della dimensione dell'input.
    * <span style="color:gold">One-sided error</span> perché l'algoritmo può sbagliare solo quando risponde False (e la risposta esatta è True).

#### <span style="color:cyan">Un algoritmo non "probability-bounded"</span>
* Un semplice algoritmo Monte Carlo one-sided error è la seguente procedura per determinare <span style="color:gold">se un numero intero $n$ dispari è composto</span>, cioè se può essere espresso come prodotto di due numeri diversi da 1 e da $n$ stesso.

1. Dato $n$, scegli a caso un numero intero <span style="color:gold">$p$ dispari</span> nell'intervallo $I=[1,n/2]$.
2. Calcola il <span style="color:gold">resto $r$</span> della divisione di $n$ per $p$
3. Se $r=0$ restituisci <span style="color:gold">True</span> (con ciò intendendo che il numero è <span style="color:gold">Composto</span>), altrimenti restituisci <span style="color:gold">False</span> (ovvero <span style="color:gold">forse Primo</span>)

* L'algoritmo è <span style="color:gold">one-sider error</span> perché può sbagliare <span style="color:gold">solo se restituisce False</span>. Se invece restituisce True il resto è 0, dunque $p$ è un divisore di $n$,che quindi è un numero composto.
* Tuttavia esso non è <span style="color:gold">probability-bounded</span>.
* Per dimostrarlo basta un argomento debole. 
* Sappiamo infatti che i divisori di un numero $n$ <span style="color:gold">vanno in coppia</span>, $x$ e $\frac{n}{x}$, di cui uno non maggiore di $\sqrt{n}$. 
* Ne consegue che i divisori di $n$ sono <span style="color:gold">al più $2\sqrt{n}$</span>. 
* Scegliendo $p$ dispari uniformemente a caso in $I$ abbiamo una probabilità che esso sia un divisore limitata da $\frac{2\sqrt{n}}{n/4}=8\frac{\sqrt{n}}{n}=\frac{8}{\sqrt{n}}$, una quantità che <span style="color:gold">tende a zero quando $n$ tende a infinito</span>.

#### <span style="color:cyan">Perché è importante limitare (anche debolmente) l'errore probabilistico</span>
* Sia <span style="color:gold">$A$</span> un algoritmo Monte Carlo per stabilire se un dato numero è composto.
* Per ogni $x\in\mathcal{Z}$, poniamo <span style="color:gold">$y_x=\mathrm{True}$</span> se il numero $x$ è composto e <span style="color:gold">$y_x=\mathrm{False}$</span> se invece $x$ è un numero primo.
* Indichiamo poi con <span style="color:gold">$A(x)$</span> la risposta fornita da $A$ su input $x$.
* La "qualifica" di $A$ come algoritmo Monte Carlo per la "compositeness" implica che:
<center>
    <span style="color:gold">$\mathrm{prob}[A(x)=\mathrm{False}|y_x=\mathrm{False}] = 1$</span>  e <span style="color:gold">$\mathrm{prob}[A(x)=\mathrm{True}|y_x=\mathrm{True}] \ge \epsilon$</span>
</center>
per un qualche valore $\epsilon>0$ opportuno.
* La probabilità che l'algoritmo sia corretto quando il numero è composto è dunque <span style="color:gold">limitata inferiormente</span>, indipedentemente dalla grandezza del numero in input.
* Supponiamo ora che risulti <span style="color:gold">$\epsilon = 0.01$</span>.
* Si può certamente obiettare che avere una probabilità di successo (su numeri composti) garantita dell'<span style="color:gold">ordine di $10^{-2}$</span> non è poi un grande risultato.
* Tuttavia se la sorgente produce bit indipendenti possiamo <span style="color:gold">aumentare la probabilità di successo</span> in modo arbitrario, come indichiamo di seguito

#### <span style="color:cyan">Run indipendenti</span>
* A partire da $A$ si costruisce un algoritmo $A'$ nel modo seguente.
* Si eseguono $N$ run identici di $A$.
* Non appena uno di tali run restituisce valore $\mathrm{True}$, anche $A'$ termina restituendo il valore $\mathrm{True}$.
* Se invece, in tutti gli $N$ run, $A$ restituisce $\mathrm{False}$, allora anche $A'$ termina restituendo il valore $\mathrm{False}$.
* Ci chiediamo quindi che cosa si può dire della correttezza di $A'$ in funzione del valore $N$.
* Se l'input è primo, ovviamente $A$ produrrà sempre il valore $\mathrm{False}$, e dunque (indipendentemente da come si è scelto $N$) anche $A'$ restituisce $\mathrm{False}$.
* Se invece il numero è composto, poiché i run sono eseguiti utilizzando sequenze di bit indipendenti, la probabilità che l'algoritmo restituisca $N$ volte False è limitata superiormente dal <span style="color:gold">prodotto delle singole probabilità</span>: $(1-0.01)^N=0.99^N$.
* Per $N=2$ risulta $0.99^2=0.98$
* Se il procedimento viene ripetuto 69 volte, con input un numero composto, la probabilità che $A$ sbagli tutte le volte è limitata dal valore $0.99^{69}<0.4998$,
* L'algoritmo $A'$ restituirà dunque la risposta corretta con <span style="color:gold">probabilità almeno $\frac{1}{2}$</span> se $N\ge 69$.
* Con valori ancora maggiori di $N$ possiamo rendere la probabilità di successo, anche nel caso in cui l'input sia un numero composto, <span style="color:gold">arbitrariamente vicina a 1</span>. 

#### <span style="color:cyan">Algoritmi Las Vegas</span>
* Un algoritmo probabilistico di tipo Las vegas restituisce <span style="color:gold">sempre la risposta corretta</span> in tempo determinato solo con una <span style="color:gold">data probabilità</span>.
* Un semplice esempio di algoritmo Las Vegas è la procedura con la quale si può generare una sorgente $\{x_i\}_{i\geq 1}$ di bit <span style="color:gold">uniformi</span> (cioè tali che $\mathrm{prob}[x_i=0]=\mathrm{prob}[x_i=1]$) e <span style="color:gold">indipendenti</span>, a partire da una sorgente $\{z_i\}_{i\geq 1}$ di bit <span style="color:gold"></span>indipendenti ma distribuiti con probabilità <span style="color:gold">non uniforme</span>: $\mathrm{prob}[z_i=0]=p, \mathrm{prob}[z_i=1]=1-p$, per un qualche valore $0<p<1$
* L'algoritmo è il seguente:
    1. Poni $i=1$
    2. Considera i bit $z_i$ e $z_{i+1}$
    3. Se $z_i=0$ e $z_{i+1}=1$ restituisci 0
    4. Se $z_i=1$ e $z_{i+1}=0$ restituisci 1
    5. Poni $i=i+2$ e ritorna al passo 2

#### <span style="color:cyan">Studio del tempo atteso</span>
* Poiché la sorgente $\{z_i\}_{i\geq 1}$ fornisce bit i.i.d, la probabilità che l'algoritmo restituisca 0 è data da <span style="color:gold">$p(1-p)$</span> e questa ovviamente è uguale a $(1-p)p$, che è la probabilità che l'algoritmo restituisca 1
* Quando dunque l'algoritmo restituisce un risultato, questo è <span style="color:gold">sempre corretto</span> (nel senso che la probabilità delle due risposte è identica)
* Tuttavia finché le coppie di bit $z_i$ e $z_{i+1}$, $i=0,2,\ldots$, sono uguali, l'algoritmo non restituisce nulla
* Quanti "cicli" dell'algoritmo dobbiamo attendere per avere il risultato?
* Se indichiamo con <span style="color:gold">$s$ il valore $1-2p(1-p)$, che è la probabilità che $z_i$ e $z_{i+1}$ siano uguali</span>, la probabilità che il risultato sia restituito al $k$-esimo tentativo è data da $s^{k-1}(1-s)$
* Siamo dunque in presenza della ben nota <span style="color:gold">distribuzione geometrica</span> il cui valore atteso è:
\begin{eqnarray*}
\sum\limits_{k=1}^{\infty}ks^{k-1}(1-s)&=&(1-s)\sum\limits_{k=1}^{\infty}ks^{k-1}\\
&=&
(1-s)\frac{d}{ds}\int_0^s\left(\sum\limits_{k=1}^{\infty}kt^{k-1}\right)dt\\
&=&
(1-s)\frac{d}{ds}\sum\limits_{k=1}^{\infty}\int_0^s kt^{k-1}dt\\
&=&
(1-s)\frac{d}{ds}\sum\limits_{k=1}^{\infty}s^k\\
&=&
(1-s)\frac{d}{ds}\left(\frac{1}{1-s}-1\right)\\
&=&
(1-s)\frac{1}{(1-s)^2}\\
&=&\frac{1}{1-s}
\end{eqnarray*}

* Chiaramente, se $p=\frac{1}{2}$, e dunque anche $s=\frac{1}{2}$, ritrovamo il <span style="color:gold">valore atteso 2</span> che è chiaramente corretto: se $\{z_i\}_{i\geq 1}$ è anche uniforme, bastano mediamente due tentativi
* Se la sorgente è invece molto "biased" verso 0 o 1, ad esempio se $p=\frac{1}{4}$, allora $s=\frac{5}{8}$ e il valore atteso è $\frac{8}{3}\approx 2.67$. Infine se $p=\frac{1}{10}$ il valor atteso diviene circa 5.56

#### <span style="color:cyan">Bit casuali e probabilità di terminazione</span>

* &Egrave; possibile anche <span style="color:gold">un'analisi "complementare"</span> del nostro semplice algoritmo Las Vegas.
* Possiamo cioè studiare la <span style="color:gold">relazione fra bit casuali</span> utilizzati (della sorgente originale $\{z_i\}_{i\geq 1}$ ovviamente) e la <span style="color:gold">probabilità che l'algoritmo termini entro un certo numero di tentativi</span>.
* Ci chiediamo quindi: qual è la <span style="color:gold">probabilità che $2k$ bit siano sufficienti</span> per ottenere il risultato, cioè il "bit uniforme"?
* Il conto è molto semplice: con $2k$ bit casuali possiamo eseguire $k$ tentativi e la probabilità che l'algoritmo termini entro $k$ tentativi è:
<center>
    $\sum\limits_{i=1}^k s^{i-1}(1-s)=(1-s)+(s-s^2)+(s^2-s^3)+\ldots+s^{k-1}-s^k=1-s^k$
</center>
dove, ricordiamo ancora una volta, $s$ è la probabilità che i due bit della sorgente considerati al generico tentativo siano uguali

#### <span style="font-style: italic; color:magenta">Esercizio</span>

* Supponiamo di voler "risparmiare" bit casuali modificando l'algoritmo nel modo seguente:
    5. Poni <span style="font-style: italic; color:gold">$i=i+1$</span> e ritorna al passo 2
* &Egrave; una buona modifica? Argomentare la risposta.

## <span style="color:gold">Teoria e algoritmi per la primalità</span> 

### <span style="color:cyan">La domanda di numeri primi</span>
* I numeri primi (o semplicemente <span style="color:gold">i primi</span>) giocano un ruolo fondamentale in tutti gli algoritmi di <span style="color:gold">crittografia asimmetrica</span>.
* Di primi <span style="color:gold">ne servono tanti</span>, da utilizzare sia in diversi protocolli sia nell'ambito dello stesso protocollo con partecipanti diversi.
* &Egrave; dunque naturale chiedersi <span style="color:gold">quanto siano abbondanti</span> i primi delle dimensioni che interessano in crittografia e <span style="color:gold">come si possa determinarli</span>.
* Sappiamo che i primi sono infiniti (si provi a dimostrarlo o a ricordare la dimostrazione); riguardo la loro abbondanza ci viene in aiuto il c.d. <span style="color:gold">Prime number theorem</span>, che descrive la distribuzione asintotica dei primi.
* Se <span style="color:gold">$\pi(n)$</span> indica il numero di <span style="color:gold">primi non maggiori di $n$</span>, vale quanto segue
<center>
    <span style="color:gold">$\pi(n)\sim\frac{n}{\ln{n}}$</span>
</center>
* Questo implica che, mediamente (e al crescere di $n$) fra i primi $n$ numeri circa <span style="color:gold">una frazione $1/\ln{n}$</span> sono primi.
* Da un punto di vista pratico, anche se con passaggio <span style="color:gold">matematicamente assai discutibile</span>, possiamo affermare che, se scegliamo a caso un numero $n$ di sufficiente grandezza, abbiamo una <span style="color:gold">probabilità circa $1/\ln{n}$</span> che esso sia primo.

#### <span style="color:cyan">Un esperimento con numeri non troppo grandi</span>

In [ ]:
# pip install pycryptodome

In [ ]:
from Crypto.Util import number
from math import log

In [ ]:
numtrials = 10000                       # Numero di esperimenti
e = 2048                                # Lunghezza in bit dei numeri generati
prob_estimate = 1.0/(e*log(2))          # Stima della probabilità
expected_num = numtrials*prob_estimate  # Valore atteso di primi

In [ ]:
# Generazione e test
numprimes = 0
for _ in range(numtrials):
    if number.isPrime(number.getRandomInteger(e)):
        numprimes += 1

In [ ]:
# Confronto con il numero atteso
print(f"Numero di primi trovati (stima, potrebbero esserci duplicati):\t{numprimes}")
print(f"Numero atteso di primi con {numtrials} esperimenti:\t\t\t{expected_num:.2f}")
print(f"Rapporto fra le due quantità:\t\t\t\t\t{numprimes/expected_num:.3f}")

#### <span style="color:cyan">Ricerca di un primo</span>

* Una <span style="color:gold">lettura "speculare"</span> del risultato precedente ci consente di affermare che, per trovare un numero primo, scegliendo a caso fra i <span style="color:gold">numeri di $n$ bit</span>, dobbiamo mediamente effettuare circa <span style="color:gold">$n\ln 2\approx 0.69n$ tentativi</span>.
* Per i numeri comunemente utilizzati in crittografia, $n$ può essere dell'ordine di qualche migliaia (<span style="color:gold">2048 o 3072 per RSA</span>, molto meno per protocolli su curve ellittiche).
* Disponendo di <span style="color:gold">algoritmi efficienti</span> per verificare se un numero è primo, il metodo comunemente utilizzato con buona efficienza consiste proprio nel <span style="color:gold">generare e verificare</span>, finché il numero generato non è primo
* Non tutti i numeri primi possono però andar bene, anche in dipendenza del protocollo utilizzato.
* Il seguente esperimento mostra come il numero medio di tentativi per trovare un primo sia in <span style="color:gold">accordo con la prescrizione teorica</span>.

In [ ]:
experiments = 1000
e = 512
avg = 0
for _ in range(experiments):
    attempts = 1
    while not number.isPrime(number.getRandomInteger(e)):
        attempts+=1
    avg+=attempts
print(f"Numero medio di tentativi effettuati:\t{float(avg)/experiments:.2f}")
print(f"Valore teorico:\t\t\t\t{0.69*e:.2f}")

#### <span style="color:cyan">Sul numero di tentativi per trovare primo sicuro</span>

In [ ]:
experiments = 5
e = 512
avg = 0
for _ in range(experiments):
    attempts = 1
    q = number.getRandomInteger(e)
    while True:
        while not number.isPrime(q):
            attempts+=1
            q = number.getRandomInteger(e)
        p = 2*q+1
        if number.isPrime(p):
            break
        else:
            attempts+=1
            q = number.getRandomInteger(e)
    avg+=attempts
print(f"Numero medio di tentativi effettuati:\t{float(avg)/experiments:.2f}")

### <span style="color:cyan">Test basato sul Piccolo Teorema di Fermat</span>

* Il piccolo teorema di Fermat afferma che, se $p$ è un numero primo, allora vale:
$$
x^{p-1}=1\mod p
$$
per ogni $x$ tale che $\mcd(x,p)=1$. In particolare, quindi, il teorema vale per ogni $x$ positivo e minore di $p$.

* Se $n$ non è un numero primo, è possibile che risulti tanto $x^{n-1}\mod n=1$ quanto $x^{n-1}\mod n\neq 1$.
* In particolare, se $x$ è composto e risulta $x^{n-1}\mod n=1$, $n$ è detto <span style="color:gold">pseudo-primo base-$x$</span>.
* L'esistenza di pseudo-primi impedisce di poter utilizzare il Piccolo Teorema di Fermat come <span style="color:gold">criterio di primalità</span>.
* Ad esempio, se non esistessero pseudo-primi base-2 basterebbe calcolare $2^{p-1}\mod p$ e verificare se il risultato è 1 o meno.
* Il fatto importante è che gli <span style="color:gold">pseudo-primi base-2</span> sono comunque pochi e tendono ad essere "straordinariamente pochi" quanto più la grandezza aumenta.
* Stime abbastanze precise ne limitano la frequenza, fra i numeri di 512 bit, a <span style="color:gold">circa 1 su $10^{20}$</span>.

#### <span style="color:cyan">Un semplice test di primalità per numeri scelti a caso</span>

* L'ultima osservazione suggerisce che, <span style="color:gold">se $n$ è scelto a caso</span> e se si usa 2 come base, è quanto meno "improbabile" che si vada a cadere su un numero pseudo-primo base 2.
* L'idea può quindi facilemente tradotta nel sequente "algoritmo".
* Si noti (qui e nel seguito) l'importazione di funzioni dal package <span style="font-style: italic; color:gold">ACLIB</span>, che contiene un po' di programmi Python usati (se non proprio spiegati) a lezione.

In [ ]:
from ACLIB.utils import modexp
def FLT_primality(n):
    "Fermat's Little Theorem primality test"
    return modexp(2,n-1,n) == 1

* Si presti bene attenzione e si rifletta sulle caratteristiche di questo test.
* Si tratta certamente di un algoritmo <span style="color:gold">one-sided error</span>, ma non di un algoritmo <span style="color:gold">Monte Carlo</span>, perché seimplicemente non effettua alcuna scelta casuale.
* Se dunque non sappiamo come è scelto l'input, non possiamo fare alcuna affermazione probabilistica! E pure se gli pseudo-primi base-2 sono rari, non possiamo escludere che il meccanismo di scelta vada a considerare proprio <span style="color:gold">questi più frequentemente</span>.
* Per questa ragione, se il risultato è True, non ha alcun senso ripetere il test per cercare di ridurre la probabilità di errore.

* Facciamo ora una piccola verifica sull'<span style="color:gold">affidabilità del metodo</span>, con numeri casuali di grandezza relativamente piccola.
* Al riguardo, riporto qui il codice per la funzione esponenziale modulare (e, conseguentemente, anche il codice per il prodotto modulare) vista nel precedente notebook.

In [ ]:
from Crypto.Util import number
numerrors = 0
e = 50   # Numeri di e bit
numexp = 50000
for _ in range(numexp):
    n = number.getRandomInteger(e)|1    # n deve essere dispari
    if FLT_primality(n) != number.isPrime(n):
        numerrors += 1
print(numerrors)

#### <span style="color:magenta">Numeri di Carmichael</span>

* Si potrebbe pensare di migliorare il test basato sul Piccolo Teorema di Fermat provando <span style="color:gold">più di una base</span> e non solo 2.
* Ad esempio potremmo provare, oltre al valore 2, anche un'altra base $a\in\zn^*$ scelta a caso.
* Questa è certamente una buona idea, come suggerisce il seguente esempio

In [ ]:
FLT_primality(561)  # 341 = 11*31

In [ ]:
modexp(3,340,341) == 1

* 341 è il più piccolo <span style="color:gold">pseudo-primo base 2</span> ma evidentemente non è uno pseudo-primo base 3.
* Possiamo quindi incorporare l'idea in un test più "sofisticato".

In [ ]:
from random import randint
from ACLIB.utils import Euclid
def improved_FLT_primality(n):
    "Improved Fermat's Little Theorem primality test."
    a = randint(3,n-1)
    if Euclid(a,n)>1:
        return False
    return (modexp(2,n-1,n) == 1) and (modexp(a,n-1,n) == 1)

In [ ]:
improved_FLT_primality(341)

* Il nuovo test è certamente <span style="color:gold">più efficace</span> per almeno due motivi.
    1.  Perché, se $n$ è composto, con probabilità non nulla il calcolo del MCD fra $n$ e $a$  lo rivela;
    2. Perché $n$ potrebbe essere psedo-primo base 2 ma non pseudo-primo base $a$
* Se il test restituisce True (suggerendo che il numero sia primo=) "può", in linea di principio può avere senso <span style="color:gold">ripeterlo</span> perché esso effettua una scelta casuale e questo può portare a risultati differenti proprio per le ragioni 1. e 2. esposte sopra.
* Esistono però numeri $n$ composti per i quali, <span style="color:gold">qualsiasi sia la base $a$ t.c. $\mcd(a,n)=1$, risulta $a^{n-1}\mod n=1$</span>.
* In altri termini, esistono <span style="color:gold">pseudo-primi base $a$</span> per ogni $a$ t.c. $\mcd(a,n)=1$.
* Questi numeri sono chiamati <span style="color:gold">(numeri) di Carmichael</span>.
* Se $n$ è un numero di Carmichael, qualora la base $a$ generata passi il test di Euclide, il test effettuato nell'ultima riga è <span style="color:gold">totalmene inutile</span>.
* I numeri di Carmichael sono <span style="color:gold">estrememente rari</span> (certamente più rari degli pseudo-primi base 2).
* Ce n'è però una relativa <span style="color:gold">abbondanza fra i numeri "piccoli"</span>.
* I primi 5 sono: <span style="color:gold">561, 1105, 1729, 2465 e 2821</span>.

In [ ]:
from ACLIB.utils import factorize

In [ ]:
factorize(2821)

In [ ]:
# Proviamo il test per ogni i t.c. MCD(i,n)=1
n = 1105
for i in range(2,n):
    if Euclid(i,n)==1:
        if modexp(i,n-1,n) != 1:
            print(f"{n} non è un numero di Carmichael")
print(f"{n} è primo oppure è un numero di Carmichael")

* Gli algoritmi utilizzati in pratica sono "veri" algoritmi probabilistici di tipo <span style="color:gold">Monte Carlo</span>.
* Questo vuol dire che possono essere ripetuti per <span style="color:gold">"abbattere" la probabilità di errore</span>.
* Si tratta di algoritmi la cui implementazione è quasi sempre <span style="color:gold">semplice</span>.
* La difficoltà risiede, anche in questo caso "quasi sempre", nella dimostrazione che la probabilità di errore (one-sided) è effettivamente <span style="color:gold">limitata da una costante</span>.
* Per gli studenti interessati, rimando ad una semplice ricerca con le parole chiave Rabin-Miller e Solovay-Strassen (dal nome dei ricercatori che li hanno ideati).

### <span style="color:cyan">Residui quadratici e test di primalità di Solovay-Strassen</span>

* Un elemento $a\in\zn$ è detto <span style="color:gold">residuo quadratico</span> (modulo $n$) se esiste $x\in\zn$ tale che $a=x^2\mod n$.
* Elenchiamo, senza dimostrazione, un po' di <span style="color:gold">proprietà dei residui quadratici</span> in $\zn^*$.
    1. Se $a$ è un r.q., allora <span style="color:gold">anche $a^{-1}\mod n$ è un r.q.</span>
    2. L'insieme dei r.q. forma un <span style="color:gold">sottogruppo di $\zn^*$</span>, che indicheremo con <span style="color:gold">$Q_n$</span>.
    3. Se $n$ è primo, la cardinalità di $Q_n$ è esattamente $|\zn^*|/2=\frac{n-1}{2}$ ed esso è formato da tutte le <span style="color:gold">potenze pari di un generatore $g$</span>.

In [ ]:
# Esempio, i residui quadratici in Z*_23
Q = {(i**2)%23 for i in range(0,23)}
Q

#### <span style="color:cyan">Simbolo di Legendre</span>

* Se $p$ è un numero primo e $a\in\zp$, il <span style="color:gold">simbolo di Legendre</span> $\left(\frac{a}{p}\right)$ è la funzione così definita
$$
\left(\frac{a}{p}\right) = \left\{\begin{array}{lcl}
0&&\mathrm{se}\ p\ \mathrm{divide}\ a\\
1&&\mathrm{se}\ a\ \mathrm{è\ un\ r.q.\ mod\ }p\\
-1&&\mathrm{se}\ a\ \mathrm{non\ è\ un\ r.q.\ mod\ }p\\
\end{array}\right.
$$
* C'è un modo semplice per <span style="color:gold">calcolare il simbolo di Legendre</span>, che è dato dal seguente <span style="color:gold">criterio di Eulero</span>
$$
\left(\frac{a}{p}\right)=a^{\left(\frac{p-1}{2}\right)}\mod p
$$
che quindi riduce il problema al calcolo di un'<span style="color:gold">esponenziale modulare</span>.
* Il criterio di Eulero consente facilmente di dimostrare che:
$$
\left(\frac{ab}{p}\right) = \left(\frac{a}{p}\right)\left(\frac{b}{p}\right)
$$
* &Egrave; inoltre immediato verificare che, <span style="color:gold">se $a\equiv b\ (\mod p)$</span>, allora
$$
\left(\frac{a}{p}\right)=\left(\frac{b}{p}\right)
$$
* Più difficili da dimostrare sono le seguenti proprietà:
    1. Legge di <span style="color:gold">reciprocità quadratica</span>. Se $p$ e $q$ sono primi dispari, vale:
    $$
    \left(\frac{q}{p}\right)\left(\frac{p}{q}\right)=(-1)^{\frac{(p-1)(q-1)}{4}}
    $$
    2. Criterio affinché <span style="color:gold">2 sia un r.q. modulo $p$</span>
    $$
    \left(\frac{2}{p}\right)=(-1)^{\frac{p^2-1}{8}}
    $$
    da cui risulta che $2$ è un r.q. se e solo se $p\equiv 1,7\ (\mod 8)$, cioè se <span style="color:gold">$p=8k+1$</span> oppure <span style="color:gold">$p=8k+7$</span>, per un qualche intero $k$.
    3. Criterio affinché <span style="color:gold">5 sia un r.q. modulo $p$</span> è che $p\equiv 1,4\ (\mod 5)$.
* 2 e 5 ci interessano da vicino in quanto sono utilizzati come generatori nell'implementazione <span style="color:gold">OPENSSL</span> del protocollo di Diffie e Hellman.

#### <span style="font-style: italic; color:cyan">Simbolo di Jacobi</span>

* Il <span style="font-style: italic; color:gold">simbolo di Jacobi</span> generalizza il simbolo di Legendre al caso dei numeri composti e per esso si usa la <span style="font-style: italic; color:gold">stessa notazione</span>
* Sia dunque $n$ un intero dispari e sia <span style="font-style: italic; color:gold">$n=p_1\cdot p_2\cdot\ldots\cdot p_k$ la sua scomposizione in fattori primi</span>, non necessariamente distinti. Per ogni intero non negativo $a$ definiamo
$$
\left(\frac{a}{n}\right)=\prod_{i=1}^k \left(\frac{a}{p_i}\right)
$$
* Si noti dunque che il simbolo di Jacobi <span style="font-style: italic; color:gold">coincide</span> con il simbolo di Legendre se $n$ è primo.
* Il seguente semplice esempio mostra tuttavia che se, se $n$ è composto, la condizione $\left(\frac{a}{n}\right)=1$ <span style="color:gold">non assicura</span> che $a$ sia un residuo quadratico modulo $n$:
$$
\left(\frac{2}{15}\right)=\left(\frac{2}{5}\right)\left(\frac{2}{3}\right)=(-1)(-1)=1
$$
* &Egrave; vero invece che se $\left(\frac{a}{n}\right)=-1$ allora $a$ <span style="color:gold">non è</span> un residuo quadratico.
* Se viceversa sappiamo che $a$ è un residuo quadratico modulo $n$, allora può succedere che $\left(\frac{a}{n}\right)=0$, ma solo nel caso in cui $\mathrm{MCD}(a,n)>1$, altrimenti $\left(\frac{a}{n}\right)=1$

* Il simbolo di Jacobi è dunque un valore nell'insieme $\{0,1,-1\}$ e gode di <span style="font-style: italic; color:gold">molte delle proprietà "di calcolo"</span> del simbolo di Legendre
    1. Proprietà <span style="font-style: italic; color:gold">moltiplicativa</span>:
    $$
    \left(\frac{ab}{n}\right) = \left(\frac{a}{n}\right)\left(\frac{b}{n}\right)
    $$
    2. Se <span style="font-style: italic; color:gold">$a\equiv b\ (\mod n)$</span></span>, allora
    $$
    \left(\frac{a}{n}\right)=\left(\frac{b}{n}\right)
    $$
    3. Se $n$ ed $m$ sono <span style="font-style: italic; color:gold">interi dispari</span>:
    $$
    \left(\frac{m}{n}\right) = \left\{\begin{array}{lcl}
    -\left(\frac{n}{m}\right)&&\mathrm{se}\ n,m\equiv 3\ (\mod 4)\\
    \\
    \left(\frac{n}{m}\right)&&\mathrm{altrimenti}
    \end{array}\right.
    $$
    4. Vale inoltre:
    $$
    \left(\frac{2}{n}\right)=1\quad\mathrm{sse\ }n\equiv 1,7\ (\mod 8)
    $$
* Il simbolo di Jacobi, tuttavia, <span style="font-style: italic; color:gold">non può essere preso come criterio</span> per stabilire se un numero $a$ è un r.q. modulo $n$.

#### <span style="font-style: italic; color:cyan">Calcolo del simbolo di Jacobi</span>
* Le proprietà elencate in precedenza consentono di calcolare il simbolo di Jacobi $\left(\frac{a}{n}\right)$ <span style="font-style: italic; color:gold">senza dover fattorizzare $n$</span>, requisito questo di importanza decisiva.

In [ ]:
from ACLIB.utils import Euclid,modexp
def jacobiS(a,n):
    '''Computes the Jacobi simbol (a/n). It is assumed (in the top leve call)
       the n is an odd positive integer'''
    if Euclid(a,n)>1:
        return 0
    if a==1:       # By definition, as the product of Legendre symbols
        return 1
    if a==2:       # Apply property 4 above
        return 1 if n%8==1 or n%8==7 else -1
    if a%2==0:     # Apply property 1 above
        return jacobiS(2,n)*jacobiS(a//2,n)
    if a>n:        # Apply property 2 above
        return jacobiS(a%n,n)
    if a%4==3 and n%4==3:     # Otherwise, apply property 3 above, checking for the case 
        return -jacobiS(n,a)  # a=n=3 (mod 4)
    return jacobiS(n,a)

* Provare a dimostrare che la precedente funzione termina sempre (e dunque è corretta perché se viene restituito un valore questo riflette le proprietà note
* La dimostrazione è solo un <span style="color:gold">fatto di logica</span>, assumendo vere le proprietà sopra elencate

In [ ]:
jacobiS(756479,1298351)

### <span style="font-style: italic; color:cyan">Test di primalità di Solovay-Strassen</span>

* Se $n$ è un numero primo sappiamo che vale l'uguaglianza 
$$
\left(\frac{a}{n}\right)= a^{\frac{n-1}{2}}\ \mod\ n
$$
* Nel caso l'uguaglianza non valga possiamo quindi certamente dire che il numero è composto. 
che invece sappiamo <span style="font-style: italic; color:gold">valere sempre se $n$ è primo</span>.
* Cosideriamo ancora, come esempio, il caso di $n=15$

* Ricordiamo che, <span style="color:gold">se $n$ è primo</span>, vale l'uguaglianza 
$$
\left(\frac{a}{n}\right)= a^{\frac{n-1}{2}}\ \mod\ n
$$
* Nel caso l'uguaglianza non valga possiamo quindi certamente dire che il numero è <span style="color:gold">composto</span>.
* &Egrave; chiaro che, se non sappiamo che $n$ è primo (e ovviamente non lo sappiamo, perché è ciò che vogliamo determinare) dobbiamo calcolare il <span style="color:gold">simbolo di Jacobi</span>, che vale per $n$ qualsiasi (primo o composto)

* Vediamo un esperimento che suggerisce perché il test di Solovay-Strassen, essenzialmente basato su questa osservazione, si può progettare come <span style="color:gold">algoritmo Monte Carlo</span>

In [ ]:
# totient è la funzione di Eulero: totient(n) calcola il numero di numeri più piccoli di n
# e coprimi con n (1 incluso). E' l'ordina del gruppo Z*_n
from sympy.ntheory.factor_ import totient

In [ ]:
while ((n := number.getRandomInteger(10)) and (number.isPrime(n) or n%2==0)): pass
print(f"Abbiamo generato {n}")
# Abbiamo generato un numero composto dispari
# Andiamo ora a verificare quanti numeri a in Z*_n "ingannano" il test di Eulero
count=0
order = totient(n) 
for a in range(1,n):
    if Euclid(a,n)>1: continue
    J = jacobiS(a,n)%n
    E = modexp(a,(n-1)>>1,n)
    if (J==E):
        count+=1
        #print(a)
print(order,count,order%count)

* Come si può vedere, l'uguaglianza non vale quasi per tutti in numeri in $\mathbf{Z}_{n}$
* Un numero composto $n$ tale che l'uguaglianza vale per un certo intero $a$ è detto <span style="font-style: italic; color:gold">pseudo-primo di Eulero rispetto alla base $a$</span>.
* L'algoritmo di Solovay e Strassen si basa sul fatto che un numero $n$ composto non è pseudo-primo di Eulero per tutte le possibili basi $a$. 
* Una valore $a$ per cui $\mcd(a,n)>1$ o tale per cui risulta $\left(\frac{a}{n}\right)\neq  a^{\frac{n-1}{2}}\ \mod\ n$ viene detto <span style="font-style: italic; color:gold">testimone della non primalità di $n$</span>

#### <span style="font-style: italic; color:cyan">Pseudo codice dell'algoritmo</span>
1. Dato $n>2$ dispari, scegli $a\in\{1,2,3,\ldots,n-1\}$ uniformemente a caso
2. Calcola $m=\mcd(a,n)$ e verifica se $m>1$; in caso affermativo restituisci $\mathtt{composto}$
3. Altrimenti calcola $J = \left(\frac{a}{n}\right)$ e $P = a^{\frac{n-1}{2}}\ \mod\ n$
4. Se $P\neq J$ restituisci <span style="color:gold">$\mathtt{composto}$</span>
5. Altrimenti restituisci <span style="color:gold">$\mathtt{probabilmente\ primo}$</span>

#### <span style="font-style: italic; color:cyan">Proprietà dell'algoritmo (non dimostrate)</span>
* Se il numero è primo, l'algoritmo restituisce sempre la <span style="font-style: italic; color:gold">risposta corretta</span> perché i test di riga 2 e 4 sono sempre falsi e dunque l'output è $\mathtt{probabilmente\ primo}$
* Tuttavia, se $n$ è composto, l'algoritmo <span style="font-style: italic; color:gold">può restituire la risposta errata</span> $\mathtt{probabilmente\ primo}$ nel caso in cui $n$ sia pseudo-primo di Eulero rispetto alla base $a$
* &Egrave; però possibile dimostrare che, per ogni intero composto dispari $n$, ci sono <span style="color:gold">al più $\frac{n-1}{2}$ interi</span> $a$ tale che $n$ è uno pseudo-primo di Eulero rispetto ad $a$
* Ne consegue che almeno $\frac{n-1}{2}$ numeri sono testimoni della non primalità e dunque che l'algoritmo erra (su input un numero composto) con <span style="font-style: italic; color:gold">probabilità non superiore a $\frac{1}{2}$</span>

### <span style="color:gold">Una funzione candidato trapdoor</span> 

### <span style="color:cyan">Fattorizzazione di interi</span>
* Sul problema di fattorizzare efficientemente numeri composti è basata, in particolare, la "forza" del <span style="color:gold">protocollo RSA</span> per cifratura e firma digitale
* Dato un numero composto $n$, l'algoritmo brute-force prova <span style="color:gold">tutti i potenziali divisori $d\leq \sqrt{n}$</span> e impiega chiaramente tempo esponenziale nella lunghezza di $n$
* Per i numeri in gioco nella crittografia, si tratta di un approccio destinato al fallimento
* Esistono però algoritmi molto più efficienti, non di costo polinomiale ma neppure esponenziale
* Il migliore algoritmo noto è il <span style="color:gold">General Number Field Sieve</span>, di cui si "stima" in modo euristico che la complessità sia una funzione del tipo $G(n)=e^{c\sqrt[3]{(\ln n)(\ln\ln n)^2}}$, con $c<3$ costante
* Una funzione del tipo $(1+c)^{d\cdot n}$, per ogni $c,d>0$ costanti, cresce <span style="color:gold">più velocemente rispetto a $G(n)$</span> e questo fornisce una spiegazione del perché RSA richieda numeri molto elevati (fattori di almeno 2048 bit)
* In generale, gli algoritmi di fattorizzazione sub-exponenziali sono piuttosto complessi e non li discuteremo ulteriormente
* Presenteremo invece l'algoritmo di fattorizzazione <span style="color:gold">$\rho$ di Pollard</span>, che (non a caso) ha lo stesso nome dell'algoritmo per trovare collisioni di funzioni hash che vedremo.
* Il pregio dell'algoritmo è la semplicità, il difetto è che richiede un numero di passi che è <span style="color:gold">ancora esponenziale nella lunghezza di $n$</span>, sebbene l'esponente possa essere <span style="color:gold">sostanzialmente minore di $n$</span>

### <span style="color:cyan">Algoritmo di fattorizzazione $\rho$ di Pollard</span>

* L'algoritmo produce, a partire da un valore $x_1\in\zn$, una successione di valori $x_{j+1}=f(x_j)\mod n$, $j=1,2,\ldots$ che, almeno idealmente, dovrebbe esibire le proprietà di una <span style="color:gold">sequenza casuale</span> che tuttavia, da un certo punto in poi, inizierà a <span style="color:gold">ripetersi</span>
* <span style="color:gold">Ogni tanto</span>, uno dei valori calcolati viene salvato ed utilizzato nelle <span style="color:gold">successive iterazioni</span> (fino al salvataggio successivo) per verificare se è stato determinato un fattore di $n$
* Le iterate salvate sono quelle di indice $i=2^t$, $t=0,1,2,\ldots$
* Ogni iterata salvata viene dunque utlizzata nei test per un numero di passi che <span style="color:gold">raddoppia</span> ad ogni successivo salvataggio
* Presentiamo prima lo pseudocodice e poi un'implementazione in Python
* Risultati <span style="color:gold">in linea con le aspettative teoriche</span> sono tipicamente ottenuti utilizzando la seguente funzione
$$
f(x) = (x^2-1)\mod n
$$
di cui anche qui facciamo uso

#### <span style="color:cyan">Algoritmo $\rho$ di Pollard per la fattorizzazione di interi</span>
1. $j=1$, $i=2$  &nbsp;&nbsp;/* $j$ indica l'iterata corrente, $i$ è l'indice del successivo salvataggio */
2. $y = x_1 = r\in\zn$ &nbsp;&nbsp;/* y è il valore salvato */
3. while True
4. &nbsp;&nbsp;&nbsp;&nbsp;$x_{j+1}=(x_j^2-1)\mod n$
5. &nbsp;&nbsp;&nbsp;&nbsp;$j = j+1$
6. &nbsp;&nbsp;&nbsp;&nbsp;$m=\mcd(y-x_j,n)$
7. &nbsp;&nbsp;&nbsp;&nbsp;if $m>1$ and $m\neq n$
7. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;print($m$); break
8. &nbsp;&nbsp;&nbsp;&nbsp;if $j==i$ &nbsp;&nbsp;/* Controlliamo se siamo arrivati al prossimo salvataggio */
9. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$i=2\cdot i$
10. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$y=x_j$

#### <span style="color:cyan">Osservazioni "semplici"</span>
* &Egrave; di facile constatazione il fatto che l'algoritmo o non termina mai oppure, se termina, restituisce effettivamente un <span style="color:gold">fattore non banale di $n$</span> 
* Come secondo step (per comprendere l'idea elegante che sta alla base dell'algoritmo), <span style="color:gold">analizziamo il test</span> che porta eventualmente alla scoperta di un tale fattore, indicato con $m$ nello pseudo-codice
* Evidentemente, in quanto $m=\mcd(y-x_j,n)$, esso divide anche la differenza $y-x_j$ 
* Detto in altri termini, questo vuol dire <span style="color:gold">$y$ e $x_j$ sono congrui modulo $m$</span>
* Scopo dell'algoritmo è dunque di andare a cercare coppie di iterate che siano diverse ma <span style="color:gold">congrue modulo un divisore di $n$</span>
* La parte "geniale" dell'algoritmo è la strategia con cui viene effettuata la ricerca, ovvero <span style="color:gold">tenendo fissa una delle due iterate nel confronto</span> (la $y$ dello pseudo-codice) per periodi <span style="color:gold">via via più lunghi</span> 

#### <span style="color:cyan">Implementazione Python e qualche esperimento</span>
* L'implementazione è data come <span style="color:gold">funzione</span> che (eventualmente) restituisce un fattore non banale di $n$

In [ ]:
from ACLIB.utils import Euclid, rand
def pollard_rho(n):
    '''Implements the Pollard's rho heuristics and (possibly)
       returns a non-trivial divisor of n.
       Strictly follows the description in 
       Cormen et al. Introduction to Algorithms, Third ed. 
    '''
    i,j = 2,1
    xprec = rand(n)  # xprec = x_1
    y = xprec        # Salvataggio del primo valore
    while True:
        j += 1
        x = (xprec*xprec - 1)%n            # calcolo iterata successiva
        if (p:=Euclid(y-x,n))!=1 and p!=n:
            return p
        if j==i:
            y = x
            i *= 2
        xprec = x

In [ ]:
n = (1097**5)*12983*(397**2); n

In [ ]:
m = pollard_rho(n); m

In [ ]:
n //= m; n

In [ ]:
pollard_rho(n)

In [ ]:
n = 13**2*7**6*17**3; n

In [ ]:
pollard_rho(n)

In [ ]:
from ACLIB.utils import getprime

In [ ]:
p = getprime(10**12);p

In [ ]:
q = getprime(10**12);q

In [ ]:
n = p*q; n

In [ ]:
pollard_rho(n)

#### <span style="color:cyan">La strategia spiegata</span>

* Per ogni possibile fattore $q$ di $n$ esiste una sorta di <span style="color:gold">"successione ombra" $\{z_i\}_{i\geq 1}$</span>, ottenuta <span style="color:gold">riducendo modulo $q$</span> ogni termine $x_i$ della successione effettivamente costruita
* Naturalmente, non conoscendo i fattori (che sono proprio ciò che andiamo a cercare) noi tali successioni non le conosciamo, ma questo non importa
* Ciò che importa sono le proprietà che di esse possiamo postulare e che, come vedremo, <span style="color:gold">dicono qualcosa</span> della successione $\{x_i\}_{i\geq 1}$
* Concentriamoci su una tale successione, per un qualche <span style="color:gold">fattore $q$ di $n$</span>
* Ciò che possiamo subito dimostrare è che la successione ombra ha la <span style="color:gold">stessa struttura</span> della $\{x_i\}_{i\geq 1}$
* Al riguardo, utilizziamo la seguente proprietà, di facile verifica. Per ogni intero $x$, vale
$$
(x\mod n)\mod q = x\mod q
$$
qualora <span style="color:gold">$q$ sia un divisore di $n$</span> (basta usare la definizione di resto)

* Possiamo in tal caso scrivere:
\begin{eqnarray*}
z_{i+1}&=&x_{i+1}\mod q\\
&=&((x_i^2-1)\mod n)\mod q\\
&=&(x_i^2-1)\mod q\\
&=&((x_i\mod q)^2-1)\mod q\\
&=&(z_i^2-1)\mod q
\end{eqnarray*}

* Ogni passo dell'algoritmo ha quindi una identica controparte nei valori che abbiamo definito "ombra". In particolare possiamo affermare che:
    1. Nell'ipotesi di comportamento realmente casuale, anche la successione ombra è caratterizzata da una sorta di "antiperiodo" (il <span style="color:gold">gambo del $\rho$</span>) e un periodo (la <span style="color:gold">testa</span>) le cui lunghezze (indicate con $\lambda$ e $\sigma$)  soddisfano <span style="color:gold">$\lambda,\sigma=O(\sqrt{q})$</span>
    2. Con la strategia di salvare i valori ad intervalli sempre più grandi abbiamo la certezza che, per un certo valore dell'indice $i$, <span style="color:gold">il valore ombra $z_{2^i}$ salvato è dentro la testa del $\rho$</span>; e ci aspettiamo che risulti $2^i=O(\sqrt{q})$
    3. Sempre per la stessa ragione, per un qualche indice $i$ maggiore o uguale al precedente, la quantità <span style="color:gold">$2^i$ è maggiore $\sigma$</span> (e dunque, ancora, $2^i=O(\sqrt{q})$)
    4. Questo però implica che esiste $j>2^i, j\leq 2^{i+1}$ tale che <span style="color:gold">$z_j=z_{2^i}$</span>
* A questo punto, però, ricordiamo che $z_i=x_i\mod q$, per ogni $i$, e dunque, nella successione originale
$$
x_j \mod q=x_{2^i}\mod q
$$
e quindi che <span style="color:gold">$x_j-x_{2^i}$ è un divisore di $q$ e dunque di $n$</span>
* La seguente figura (tratta dal libro <span style="color:gold">Cormen et al. Introduction to Algorithms, Third ed.</span>) illustra la situazione per il particolare <span style="color:gold">valore $n=1387=19\cdot 73$</span>

<center>
    <img src="pollard_rho.jpg" width="800">
</center>

In [ ]:
(84-814)%73

In [ ]:
# Con riferimento alla successione mod 73
(31**2-1)%73 == 11 and 814%73 == 11  # I due modi di ottenere z_8=11

In [ ]:
# Poiché z_8 = z_12, risulta che x_8-x_12 è divisibile per 73
Euclid(814-84,1387)

#### <span style="color:cyan">Cosa può andare storto?</span>
* Potrebbe succedere che i valori di $\lambda$ e $\sigma$ per più di una sequenza ombra siano tali che la "scoperta" di due iterate identiche avvenga <span style="color:gold">nello stesso istante</span>
* Per esemplificare il caso "meno improbabile", se $n=p\cdot q$, con $p$ e $q$ primi, la concomitanza della scoperta fa sì che le corrispondenti iterate $x_j$ e $x_{2^i}$ siano <span style="color:gold">congruenti modulo $p$ e modulo $q$</span>
* Poiché $p$ e $q$ sono primi, questo implica che $x_j$ e $x_{2^i}$ sono anche congrui modulo $n$ e dunque che $\mcd(x_{2^i},x_j)=n$
* Questo ovviamente <span style="color:gold">non aiuta</span>
* Nel caso di scomposizione $n=p^k$, può "semplicemente" succedere che quando $z_{2^i}=z_j$ <span style="color:gold">risulti anche $x_{2^i}=x_j$</span>
* Anche in questo caso non viene scoperto nulla

In [ ]:
pollard_rho(7**2)

In [ ]:
pollard_rho(23**3)

#### <span style="color:cyan">Efficienza</span>
* Avendo ben presente il <span style="color:gold">carattere euristico</span> dell'analisi relativa alla lunghezza di $\lambda$ e $\sigma$ per le varie sequenze ombra, è ragionevole attendersi che il numero di operazioni necessarie per trovare <span style="color:gold">il più piccolo fattore primo $p$ di $n$ sia $O(\sqrt{p})$ </span>
* Poiché $p<\sqrt{n}$, la stima complessiva diviene $O(\sqrt[4]{n})$ operazioni su numeri di $O(\log n)$ bit 

### <span style="color:magenta">Fattorizzazione completa</span>
* Possiamo ora utilizzare la funzione pollard_rho per <span style="color:gold">tentare la fattorizzazione completa</span> di $n$
* Dobbiamo però prima di tutto decidere <span style="color:gold">come rappresentare le fattorizzazioni</span>

In [ ]:
class primefact(dict):
    '''Simple class to represent factorizations'''
    def __new__(cls,*args):
        return super().__new__(cls,{})
    def __init__(self,*args):
        assert not len(args)&1, "Wrong arguments"
        for i in range(0,len(args),2):
            self[args[i]]=args[i+1]
    def primes(self):
        return set(self.keys())
    def __repr__(self):
        return " * ".join([str(p) if e==1 else str(p)+"**"+str(e) for p,e in self.items()])
    def merge(self,other):
        for k in self.primes().union(other.primes()):
            self[k]=self.get(k,0)+other.get(k,0)

In [ ]:
a = primefact(2,2,3,4);a

In [ ]:
a.primes()

In [ ]:
b = primefact()

In [ ]:
from ACLIB.utils import isPrime, BSGS
from math import sqrt
def BruteForceFact(n):
    '''Computes the prime facorization using brute force.
       Use carefully for small (and odd) values of n 
    '''
    assert n&1, "n must be odd"
    F = primefact()
    while n>1 and not isPrime(n):
        for i in range(3,int(sqrt(n))+1,2):
            if n%i==0:
                F.merge(primefact(i,1))
                n=n//i
                break
    if n>1:
        F.merge(primefact(n,1))
    return F

def factorize(n,nmax=10**6):
    '''Return the prime factorization of n'''
    def fact(n):
        '''Inner function that actually does the heavy job.
           Uses brute force for small n and Pollard's rho heuristics otherwise.
           Not guaranteed to terminate, since pollard_rho 
           does not halt occasionally
        '''
        if n<=nmax:
            return BruteForceFact(n)
        elif isPrime(n):
            return primefact(n,1)
        f = pollard_rho(n)
        F = fact(f)
        F.merge(fact(n//f))
        return F
    # We first deal with the case n=p 2^q and compute q (p is odd)
    i = 0
    while not n&1:
        i += 1
        n = (n>>1) 
    if i==0:
        d = primefact()
    else:
        d = primefact(2,i)
    # What remains is p
    if n != 1:
        d.merge(fact(n))
    return d

In [ ]:
n = 17**4*31**3*59; n

In [ ]:
factorize(n)

In [ ]:
n=1387; factorize(n)

In [ ]:
n = 8965*10001**2; n

In [ ]:
f=factorize(n); f

In [ ]:
eval(str(f))